# Bonus Round: Exercises
Debug the following advanced snippets.

## Exercise 1

In [ ]:
def read_config(path):
    f = open(path)            # file may leak if parse() raises
    data = parse(f.read())    # assume parse may raise NameError here
    f.close()
    return data

# Run to see the problem (parse is undefined, and file isn't closed safely)
try:
    read_config("config.txt")
except Exception as e:
    print(type(e).__name__, e)

## Exercise 2

In [ ]:
import csv

def save_rows(path, rows):
    f = open(path, "w")  # missing newline and encoding and context manager
    writer = csv.writer(f)
    for r in rows:
        writer.writerow(r)
    f.close()

save_rows("out.csv", [["naïve", "café"], ["a", "b"]])
print("Wrote out.csv — check for blank lines on Windows and encoding issues.")

## Exercise 3

In [ ]:
import asyncio, time

async def fetch(i):
    time.sleep(0.1)  # blocks event loop
    return i

async def main():
    tasks = [fetch(i) for i in range(5)]
    results = asyncio.gather(*tasks)  # not awaited
    print("Finished? (but tasks may not have run)")
    return results

try:
    asyncio.run(main())
except Exception as e:
    print(type(e).__name__, e)

## Exercise 4

In [ ]:
import threading

counter = 0

def work(n):
    global counter
    for _ in range(n):
        counter += 1  # data race

threads = [threading.Thread(target=work, args=(100_000,)) for _ in range(4)]
for t in threads: t.start()
for t in threads: t.join()
print("Final counter:", counter, "Expected:", 400_000)

## Exercise 5

In [ ]:
import sqlite3
conn = sqlite3.connect(":memory:")
cur = conn.cursor()
cur.execute("CREATE TABLE users (id INT, name TEXT)")
cur.execute("INSERT INTO users VALUES (1, 'O\'Malley')")

username = "O'Malley"
query = f"SELECT id, name FROM users WHERE name = '{username}'"
print("Running:", query)  # will break on quotes and is injectable
try:
    cur.execute(query)
    print(cur.fetchone())
except Exception as e:
    print(type(e).__name__, e)

## Exercise 6

In [ ]:
from datetime import datetime, timedelta

start = datetime.now()              # naive
end = start + timedelta(hours=24)
print("Seconds diff:", (end - start).total_seconds())  # assume 86400; may be wrong over DST

## Exercise 7

In [ ]:
def process(lines_iter):
    return [s.strip() for s in lines_iter if s.strip()]

data = iter(["a\n", "\n", "b\n"])
first = process(data)
second = process(data)  # empty: iterator already consumed
print("first:", first, "second:", second)

## Exercise 8

In [ ]:
from dataclasses import dataclass

@dataclass
class Cart:
    items: list = []   # shared across instances

c1 = Cart()
c2 = Cart()
c1.items.append("x")
print("c2.items:", c2.items)  # unexpected shared list

## Exercise 9

In [ ]:
import os, logging
API_KEY = "sk_live_example_secret_value"  # pretend secret

def charge(amount_pennies):
    try:
        raise TimeoutError("simulated")
    except Exception as e:
        logging.error("Charge failed for key=%s amount=%s err=%r", API_KEY, amount_pennies, e)
        return False

print("Result:", charge(1234))